Initialize libraries
Download required packages

In [1]:
!pip install geopy
!pip install folium
!pip install geocoder
print("Installed!")

Installed!


In [2]:

# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [3]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
dataFromWiki = BeautifulSoup(raw_wikipedia_page,'lxml')
dataFromWiki

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XourOApAIDAAABMB7dwAAADH","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":949497198,"wgRevisionId":949497198,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"

Scrapping data from XML


In [4]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = dataFromWiki.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [5]:
# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [6]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [7]:
df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M2N,North York,Downsview
freq,1,24,4


Data Cleaning | Drop None rows of df and row which contains 'Not assigned' value | All "Not assigned" will be replace to 'NaN'

In [8]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [9]:
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [10]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df_2

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [11]:
print(df_2.shape)
df_2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df_2.to_csv('toronto.csv', index=False)

In [13]:
import geocoder

In [14]:
df3 = pd.read_csv('toronto.csv')
df3.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
print(df3.shape)
df3.describe()

(103, 3)


,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M2N,North York,Downsview
freq,1,24,4


In [16]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.70941386000004, -79.36309957799995]

In [28]:

# Retrieving Postal Code Co-ordinates
postal_codes = df3['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [29]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df3['Latitude'] = df_coords['Latitude']
df3['Longitude'] = df_coords['Longitude']

In [30]:
df[df3.Postalcode == 'M5G']

,Postalcode,Borough,Neighborhood
57,M5G,Downtown Toronto,Central Bay Street


In [31]:
df3.head(15)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.726881,-79.265694
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.713340,-79.284942
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.723538,-79.228353
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.696448,-79.265642


In [35]:
df3.to_csv('toronto_part2.csv',index=False)

In [36]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [55]:
df4 = pd.read_csv('toronto_part2.csv')
print(df4.shape)
df4.head()


(103, 5)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [56]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

C:\Users\karthika.gopalakrish\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


In [57]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

In [67]:
df4.head()



,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


Create Dataframe with Neighborhoods


In [68]:
# @hiddel_cell
CLIENT_ID = 'YGCBXUOJBD34RYUXGNAU5OUIEFA34BQNLMX0L4ZTII4IFYIS' # your Foursquare ID
CLIENT_SECRET = 'Q0ZZRQOCOS2HLCNJFAYCIWKTEKF4WHHSVOACXKCICFMNR3RO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [70]:
scarborough_data = df4[df4['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.726881,-79.265694


Let's Create Map

In [71]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [73]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

Get 100 venues

In [75]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Malvern / Rouge" are 43.80862623100006, -79.18991284599997.


In [77]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [78]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e95f187216785001bdda465'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c059bcd7083952134097bce',
       'name': 'SEPHORA',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'at Scarborough Town Centre',
        'lat': 43.77501688366838,
        'lng': -79.25810909472256,
       

In [79]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [80]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)


C:\Users\karthika.gopalakrish\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,SEPHORA,Cosmetics Shop,43.775017,-79.258109
1,Disney Store,Toy / Game Store,43.775537,-79.256833
2,American Eagle Outfitters,Clothing Store,43.776012,-79.258334
3,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
4,Tommy Hilfiger,Clothing Store,43.776015,-79.257369
5,DAVIDsTEA,Tea Room,43.776320,-79.258688
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Hot Topic,Clothing Store,43.775450,-79.257929
8,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662
9,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649


In [81]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Let's get veues for each neighborhood

In [84]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge


In [85]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.808626,-79.189913,Upper Rouge Trail,43.809988,-79.186147,Trail
1,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368,Fox and Fiddle,43.789082,-79.154459,Bar
2,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,Guildwood / Morningside / West Hill,43.765806,-79.185284,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
4,Guildwood / Morningside / West Hill,43.765806,-79.185284,Little Caesars Pizza,43.769046,-79.184386,Pizza Place


In [86]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,6,6,6,6,6,6
Birch Cliff / Cliffside West,4,4,4,4,4,4
Cedarbrae,3,3,3,3,3,3
Clarks Corners / Tam O'Shanter / Sullivan,7,7,7,7,7,7
Cliffside / Cliffcrest / Scarborough Village West,6,6,6,6,6,6
Dorset Park / Wexford Heights / Scarborough Town Centre,4,4,4,4,4,4
Golden Mile / Clairlea / Oakridge,9,9,9,9,9,9
Guildwood / Morningside / West Hill,20,20,20,20,20,20
Kennedy Park / Ionview / East Birchmount Park,6,6,6,6,6,6


In [87]:

print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 49 uniques categories.


In [88]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Beer Store,Big Box Store,Bistro,Breakfast Spot,Brewery,Bus Line,Bus Station,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Hobby Shop,Indian Restaurant,Intersection,Korean Restaurant,Liquor Store,Lounge,Metro Station,Mexican Restaurant,Other Great Outdoors,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Sports Bar,Supermarket,Sushi Restaurant,Thrift / Vintage Store,Trail,Wine Shop
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Rouge Hill / Port Union / Highland Creek,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
scarb_onehot.shape

(90, 50)

In [90]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Beer Store,Big Box Store,Bistro,Breakfast Spot,Brewery,Bus Line,Bus Station,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Hobby Shop,Indian Restaurant,Intersection,Korean Restaurant,Liquor Store,Lounge,Metro Station,Mexican Restaurant,Other Great Outdoors,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Sports Bar,Supermarket,Sushi Restaurant,Thrift / Vintage Store,Trail,Wine Shop
0,Agincourt,0.00,0.166667,0.000000,0.0,0.0,0.0,0.0,0.000000,0.166667,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.166667,0.000000,0.0,0.166667,0.166667,0.0,0.000000,0.00
1,Birch Cliff / Cliffside West,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00
2,Cedarbrae,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.333333,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.00
3,Clarks Corners / Tam O'Shanter / Sullivan,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.142857,0.00,0.000000,0.142857,0.0,0.0,0.0,0.0,0.00,0.000000,0.142857,0.0,0.0,0.0,0.0,0.142857,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.142857,0.142857,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00
4,Cliffside / Cliffcrest / Scarborough Village West,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.000000,0.00,0.000000,0.000000,0.0,0.166667,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.166667,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00
5,Dorset Park / Wexford Heights / Scarborough To...,0.25,0.000000,0.250000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.25
6,Golden Mile / Clairlea / Oakridge,0.00,0.000000,0.111111,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.222222,0.111111,0.0,0.111111,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.222222,0.0,0.000000,0.000000,0.111111,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,0.0,0.000000,0.000000,0.0,0.000000,0.00


LEt's get top 10 venues

In [91]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Badminton Court,Sushi Restaurant,Supermarket,Skating Rink,Shanghai Restaurant,Bus Station,Fast Food Restaurant,Discount Store,Department Store
1,Birch Cliff / Cliffside West,Pizza Place,Skating Rink,College Stadium,General Entertainment,Bank,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
2,Cedarbrae,Trail,Lounge,Construction & Landscaping,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store
3,Clarks Corners / Tam O'Shanter / Sullivan,Pharmacy,Golf Course,Convenience Store,Intersection,Shopping Mall,Coffee Shop,Pizza Place,Bus Station,Fast Food Restaurant,Discount Store
4,Cliffside / Cliffcrest / Scarborough Village West,Pharmacy,Bistro,Liquor Store,Sandwich Place,Coffee Shop,Gift Shop,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store
5,Dorset Park / Wexford Heights / Scarborough To...,Wine Shop,Brewery,Bakery,Auto Garage,Supermarket,General Entertainment,Fast Food Restaurant,Discount Store,Department Store,Convenience Store
6,Golden Mile / Clairlea / Oakridge,Intersection,Bus Line,Bus Station,Coffee Shop,Bakery,Soccer Field,Metro Station,Fried Chicken Joint,Fast Food Restaurant,Discount Store
7,Guildwood / Morningside / West Hill,Fast Food Restaurant,Pharmacy,Pizza Place,Restaurant,Fried Chicken Joint,Grocery Store,Coffee Shop,Intersection,Bus Line,Breakfast Spot
8,Kennedy Park / Ionview / East Birchmount Park,Discount Store,Chinese Restaurant,Hobby Shop,Department Store,Coffee Shop,Wine Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Convenience Store
9,Malvern / Rouge,Trail,Chinese Restaurant,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,Construction & Landscaping


Create Clusters using K Menas

In [93]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2])

In [96]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913,1,Trail,Chinese Restaurant,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,Construction & Landscaping
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368,1,Bar,Construction & Landscaping,Wine Shop,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284,1,Fast Food Restaurant,Pharmacy,Pizza Place,Restaurant,Fried Chicken Joint,Grocery Store,Coffee Shop,Intersection,Bus Line,Breakfast Spot
3,M1G,Scarborough,Woburn,43.771545,-79.218135,1,Coffee Shop,Pharmacy,Korean Restaurant,Wine Shop,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,Construction & Landscaping
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,1,Trail,Lounge,Construction & Landscaping,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725,1,Fast Food Restaurant,Indian Restaurant,Big Box Store,Sandwich Place,Wine Shop,Chinese Restaurant,Fried Chicken Joint,Discount Store,Department Store,Convenience Store
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.726881,-79.265694,1,Discount Store,Chinese Restaurant,Hobby Shop,Department Store,Coffee Shop,Wine Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Convenience Store
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.713340,-79.284942,1,Intersection,Bus Line,Bus Station,Coffee Shop,Bakery,Soccer Field,Metro Station,Fried Chicken Joint,Fast Food Restaurant,Discount Store
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.723538,-79.228353,1,Pharmacy,Bistro,Liquor Store,Sandwich Place,Coffee Shop,Gift Shop,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.696448,-79.265642,2,Pizza Place,Skating Rink,College Stadium,General Entertainment,Bank,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store


Let's examine each clusters

In [97]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,0,Auto Garage,Golf Course,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium


In [98]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Trail,Chinese Restaurant,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,Construction & Landscaping
1,Scarborough,1,Bar,Construction & Landscaping,Wine Shop,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store
2,Scarborough,1,Fast Food Restaurant,Pharmacy,Pizza Place,Restaurant,Fried Chicken Joint,Grocery Store,Coffee Shop,Intersection,Bus Line,Breakfast Spot
3,Scarborough,1,Coffee Shop,Pharmacy,Korean Restaurant,Wine Shop,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,Construction & Landscaping
4,Scarborough,1,Trail,Lounge,Construction & Landscaping,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store
5,Scarborough,1,Fast Food Restaurant,Indian Restaurant,Big Box Store,Sandwich Place,Wine Shop,Chinese Restaurant,Fried Chicken Joint,Discount Store,Department Store,Convenience Store
6,Scarborough,1,Discount Store,Chinese Restaurant,Hobby Shop,Department Store,Coffee Shop,Wine Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Convenience Store
7,Scarborough,1,Intersection,Bus Line,Bus Station,Coffee Shop,Bakery,Soccer Field,Metro Station,Fried Chicken Joint,Fast Food Restaurant,Discount Store
8,Scarborough,1,Pharmacy,Bistro,Liquor Store,Sandwich Place,Coffee Shop,Gift Shop,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store
12,Scarborough,1,Breakfast Spot,Badminton Court,Sushi Restaurant,Supermarket,Skating Rink,Shanghai Restaurant,Bus Station,Fast Food Restaurant,Discount Store,Department Store


In [99]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,3,Wine Shop,Brewery,Bakery,Auto Garage,Supermarket,General Entertainment,Fast Food Restaurant,Discount Store,Department Store,Convenience Store


In [100]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4,Pharmacy,Wine Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,Construction & Landscaping,College Stadium


In [101]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,2,Pizza Place,Skating Rink,College Stadium,General Entertainment,Bank,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store
